In [1]:
USER_KEY = "cc3948f0cd364a9f684a1fe78b9229ba"      #  Input your User key
DATA_KEY = "a71eaf04-802f-40be-93c2-5bee2548f4db"  #  Fixed Data Api key

print("USER_KEY:", USER_KEY if USER_KEY else "Not found")
print("DATA_KEY:", DATA_KEY if DATA_KEY else "Not found")

USER_KEY: cc3948f0cd364a9f684a1fe78b9229ba
DATA_KEY: a71eaf04-802f-40be-93c2-5bee2548f4db


In [2]:
import requests

root_url = f'https://aifapbt.fin.cloud.ainode.ai/'

headers = {"API-KEY": USER_KEY}
requests.get(root_url, headers=headers).json()

{'message': 'API Server is Running.',
 'status': 200,
 'User status': 'active',
 'Expires At': '2025-09-17T00:00:00',
 'Level': 1}

In [20]:
import os
strategy_name = "idiosyncratic_volatilites"

In [21]:
# Upload or Update strategy file
endpoint = 'upload/strategy'
url = root_url + endpoint

base_path = '/Users/seongmin/Desktop/neomatrix_trading/futures'
file_path = os.path.join(base_path, strategy_name, strategy_name + ".py")

params = {'tradeType':'futures', 'overwrite': True} # If overwrite is True, it will modify the existing file.

with open(file_path, "rb") as f:
    files = {"file": f}
    response = requests.post(url, headers=headers, params=params, files=files)

print("📂 Upload Response:", response.json())

📂 Upload Response: {'message': "'idiosyncratic_volatilites' Strategy Updated Successfully!"}


In [22]:
# Check strategy file upload
endpoint = 'upload/check/strategy'
url = root_url + endpoint

params = {"tradeType": "futures", "strategy_name": strategy_name}

response = requests.get(url, headers=headers, params=params)
print(response.json()['content'])

# strategy.py

import pandas as pd
import numpy as np
import statsmodels.api as sm

def strategy(df, config_dict):
    """
    Implements a long-short strategy based on asset sensitivity to systematic volatility shocks,
    inspired by "The Cross-Section of Volatility and Expected Returns".

    This adaptation is for cryptocurrency markets where a VIX index is unavailable.
    It proxies systematic volatility using the volatility of an equal-weighted crypto market index.

    The strategy is based on the paper's finding that assets with high sensitivity (high beta)
    to volatility shocks tend to have lower future returns. Therefore, this implementation
    shorts the high-beta assets and longs the low-beta assets.

    Args:
        df (pd.DataFrame): Price time-series data with datetime index and symbols as columns.
        config_dict (dict): Dictionary containing strategy settings.

    Returns:
        dict: A dictionary of weights for each symbol, e.g., {"BTCUSDT": 0.1, "ETHUSD

In [23]:
"""# Strategy configs
strategy_config_params = {
  "rebalancing_config": {
    "rebalancing_interval_hours": 72, # Rebalancing cycle (choose between 6, 12, 24, and 72 hours)
  },
  "strategy_config": {
    "beta_lookback_period": 1440,     # Lookback period in minutes for the OLS regression to calculate volatility betas. 1440 minutes equals one day.
    "volatility_lookback": 60,        # Lookback period in minutes for the rolling standard deviation to calculate the market volatility proxy.
    "quantile_threshold": 0.3         # The top and bottom quantile to select assets for shorting and longing. 0.3 means shorting the top 30% and longing the bottom 30% of assets based on volatility beta.
    }
}
"""
# Strategy configs
strategy_config_params = {
  "rebalancing_config": {             # Rebalancing settings
    "rebalancing_interval_hours": 72, ## Rebalancing cycle (choose between 6, 12, 24, and 72 hours)
  },
  "strategy_config": {
    "lookback": 60,              # Lookback period for momentum calculation
    "long_allocation_pct": 0.6,  # Proportion of capital allocated to longs
    "short_allocation_pct": 0.4  # Proportion of capital allocated to shorts
    }
}

In [24]:
# System configs
start_date_str = "2025-12-01"
end_date_str = "2025-12-30"
lookback_min = 60 # Max lookback minutes the script needs for data history
initial_capital = 10000000
leverage = 100

symbols = [
    'BTCUSDT',
    'ETHUSDT',
    'XRPUSDT',
    'SOLUSDT',
    'DOGEUSDT',
    'ADAUSDT',
    'BCHUSDT',
    'LINKUSDT',
    'XLMUSDT',
    'AVAXUSDT',
    'SHIBUSDT',
    'LTCUSDT',
    'DOTUSDT',
    'UNIUSDT',
    'PEPEUSDT',
    'AAVEUSDT',
    'APTUSDT',
    'ICPUSDT',
    'NEARUSDT',
    'ETCUSDT',
    'FETUSDT',
    'POLUSDT',
    'MKRUSDT',
    'RENDERUSDT',
    'ATOMUSDT',
    'ALGOUSDT',
    'QNTUSDT',
    'INJUSDT',
    'BONKUSDT',
    'TIAUSDT',
    'STXUSDT',
    'CVXUSDT',
    'GRTUSDT',
    'CRVUSDT',
    'AEROUSDT',
    'LDOUSDT',
    'SANDUSDT',
    'JASMYUSDT',
    'XTZUSDT',
    'MANAUSDT',
    'APEUSDT',
    'COMPUSDT',
    'HNTUSDT',
    'AXSUSDT',
    'CHZUSDT',
    '1INCHUSDT',
    'LPTUSDT',
    'SNXUSDT',
    'ROSEUSDT'
]

generate_report_flag = True

In [25]:
import json
from datetime import datetime

path = "/Users/seongmin/Desktop/neomatrix_trading/backtest_report"

request_payload = {
    "data_apikey": DATA_KEY,
    "strategy": strategy_name,
    "strategy_config": strategy_config_params,
    "start_date": start_date_str,
    "end_date": end_date_str,
    "lookback_minutes": lookback_min,
    "capital": initial_capital,
    "leverage": leverage,
    "symbols": symbols,
    "calendar": "24/7",
    "frequency": "minute",
    "generate_pyfolio_report": generate_report_flag
}

endpoint = '/run/futures/backtest'
url = root_url + endpoint

try:
    response = requests.post(url, json=request_payload, headers=headers)
    response.raise_for_status()
    print(f"\n--- Backtest execution successful (Status Code: {response.status_code}) ---")

    try:
        result_data = response.json()
        report_type = result_data.get('report_type') # Check report type

        if report_type == 'html':
            print("Report Type: HTML Report included.")
            html_content = result_data.get('html_content')
            logs = result_data.get('logs', 'No stderr logs received.')
            stdout_logs = result_data.get('stdout', 'No stdout received.')

            if html_content:
                if not os.path.exists(path):
                    os.makedirs(path)

                report_filename = os.path.join(path, f"{datetime.now().strftime('%Y-%m-%d %H:%M')}_{strategy_name}_backtest_report.html")
                try:
                    with open(report_filename, "w", encoding="utf-8") as f:
                        f.write(html_content)
                    print(f"HTML report received and saved successfully as '{report_filename}'.")
                except Exception as e:
                    print(f"ERROR: Failed to save received HTML report: {e}")
                    print("\n--- Received HTML Content (Snippet) ---")
                    print(html_content[:1000] + "...") # Output some content when saving fails
            else:
                print("WARN: Report type was 'html' but no HTML content found in response.")

            # print logging
            print("\n--- Execution Logs (stderr) ---")
            print(logs)
            if stdout_logs:
                print("\n--- Execution Output (stdout) ---")
                print(stdout_logs)

        elif report_type == 'logs_only':
            print(f"Report Type: Logs Only (Report generation skipped).")
            print(f"Message: {result_data.get('message')}")
            print("\n--- Execution Logs (stderr) ---")
            print(result_data.get('logs', 'No stderr logs received.'))
            if 'stdout' in result_data:
                 print("\n--- Execution Output (stdout) ---")
                 print(result_data.get('stdout', 'No stdout received.'))
        else:
            print(f"WARN: Received successful response with unknown report_type: '{report_type}'")
            print("\n--- Full JSON Response ---")
            print(json.dumps(result_data, indent=2, ensure_ascii=False))

    except json.JSONDecodeError:
        print("ERROR: Failed to decode JSON response from successful API call.")
        print(f"Content-Type: {response.headers.get('content-type', 'N/A')}")
        print("\n--- Received Raw Content (First 1000 chars) ---")
        print(response.text[:1000] + "...")

except requests.exceptions.Timeout:
    print(f"\n--- API call failed: Timeout) ---")
except requests.exceptions.HTTPError as e:
    print(f"\n--- API call failed: HTTP Error {e.response.status_code} ---")
    try:
        error_details = e.response.json()
        print("Error details:")
        print(json.dumps(error_details, indent=2, ensure_ascii=False))
    except json.JSONDecodeError:
        print("Error response content (Non-JSON):")
        print(e.response.text)
except requests.exceptions.RequestException as e:
    print(f"\n--- API call failure: Request Error ---")
    print(f"Error connecting to or requesting the API server ({url}): {e}")
except Exception as e:
    print(f"\n--- Unexpected error occurred ---")
    print(f"Error type: {type(e).__name__}, Content: {e}")


--- API call failed: HTTP Error 404 ---
Error details:
{
  "message": "The requested resource was not found. Please check the API URL."
}
